In [ ]:

import pandas as pd
from selenium import webdriver
import csv

In [ ]:
# Initialize the Chrome WebDriver
webframe=webdriver.Chrome()

In [ ]:
# Get search term and number of pages to scrape from the user
Search=input("please insert what you want to search")
R=int(input("please insert how many page you want"))
csv_file_name = f"{Search}.csv" # Create a CSV file name based on the search term

In [ ]:
# Initialize lists to hold job data
s=0
t_job=[]
Com_name=[]
com_place=[]
J_time=[]
j_col=[]
# Loop through each page of search results
for s in range (R) :
 
 # Build the URL for the current page and load it
 url=f"https://wuzzuf.net/search/jobs/?a=hpb&q={Search}&start={s+1}"
 webframe.get(url)

 # Extract job titles, company names, company locations, job times, and job descriptions
 title_jobs=webframe.find_elements("xpath" , '//h2[@class="css-m604qf"]')
 Company_Name=webframe.find_elements("xpath" , '//a[@class="css-17s97q8"]')
 Company_place=webframe.find_elements("xpath" , '//span[@class="css-5wys0k"]')
 Job_time=webframe.find_elements("xpath" , '//div[@class="css-do6t5g"]')
 job_col=webframe.find_elements("xpath",'//div[@class="css-y4udm8"]')

 # Append the extracted data to the respective lists
 for t in title_jobs:
   t_job.append(t.text)
 for t in Company_Name:
   Com_name.append(t.text)
 for t in Company_place:
   com_place.append(t.text)
 for t in Job_time:
   J_time.append(t.text)
 for t in job_col:
   j_col.append(t.text)
 
 s+=1  # Increment the page counter



In [ ]:
# Open the CSV file for writing and save the extracted data
with open(csv_file_name, 'w', newline='',encoding='UTF-8') as csv_file:

    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header row
    title=csv_writer.writerow(["Time","Job Title", "Company Name", "Company place", "Description"])

    # Combine the data into rows
    rows = zip(J_time,t_job,Com_name,com_place,j_col)
    
    # Iterate over the rows and write them to the CSV file
    for row in rows:
        csv_writer.writerow([elem for elem in row])
      
 
print(f"Data has been saved to {csv_file_name}")

In [ ]:
# Load the saved CSV file into a DataFrame
df=pd.read_csv(f"{csv_file_name}")
df.head()

In [ ]:
# Split the 'Description' column into two new columns based on the first newline
df_split = df['Description'].str.split('\n',n=1, expand=True)
df_split.head()

In [ ]:
# Further split the second column into 'job stat' and 'Experience'
df_split[['job stat','Experince']] = df_split[1].str.split('\n',n=1, expand=True)
df_split.drop(df_split.columns[1],axis=1,inplace=True) # Drop the now redundant column
df_split.head()


In [ ]:
# Combine the first two columns into a new 'Job Stat' column and drop the old columns
df_split['Job Stat'] = df_split[0] +' '+ df_split['job stat']
df_split.drop([0,'job stat'],axis=1,inplace=True)
df_split.head()

In [ ]:
# Split the 'Experience' column into three new columns
x = df_split['Experince'].str.split('·',n=2, expand=True)
x.head()

In [ ]:
# Drop the original 'Experience' column and concatenate the new columns with the DataFrame
df_split.drop(['Experince'],axis=1,inplace=True)
df_split=pd.concat ([df_split,x],axis=1)
df_split.head()

In [ ]:
# Combine the first two columns into a new 'Experience' column and drop the old columns
df_split['Experience'] = df_split[0] +' '+ df_split[1]
df_split.drop([0,1],axis=1,inplace=True)
df_split.head()

In [ ]:
# Split the third column into 'Categories' and 'Skills' columns
df_split[['Categories','Skills']] = df_split[2].str.split('\n',n=1, expand=True)
df_split.drop(df_split.columns[1],axis=1,inplace=True)
df_split.head()

In [ ]:
# Drop the original 'Description' column and concatenate the split DataFrame with the original DataFrame
df.drop(['Description'],axis=1,inplace=True)
df=pd.concat ([df,df_split],axis=1)
df.head()

In [ ]:
# Clean up the text in 'Company Name', 'Categories', 'Skills', and 'Experience' columns
df['Company Name'] = df['Company Name'].str.replace('-', '').str.strip()
df['Categories'] = df['Categories'].str.replace('·', '-').str.strip()
df['Skills'] = df['Skills'].str.replace('·', ' ').str.strip()
df['Experience'] = df['Experience'].str.replace('On-site', '', regex=False).str.strip()
df['Experience'] = df['Experience'].str.replace('IT/Software Development', '', regex=False).str.strip()
df['Skills'] = df['Skills'].str.replace('\n\n', '', regex=False).str.strip()
df.head()

In [ ]:
# Function to clean and order content in each cell
def clean_and_order_text(text):
    # Split by newlines and strip each part
    parts = [part.strip() for part in text.split('\n') if part.strip()]
    
    # Sort the parts alphabetically (optional, if ordering is needed)
    parts.sort()
    
    # Join the parts back into a single string with a single newline
    return '\n'.join(parts)

# Apply the function to the 'Description' column
df['Skills'] = df['Skills'].apply(clean_and_order_text)
df.head()


In [ ]:
# Save the cleaned DataFrame back to a CSV file
df.to_csv(f"{Search}.csv", index=False)